# Clickhouse vs Vertica

#### 1. Создание таблиц для работы 

#### *Clickhouse*

In [144]:
from clickhouse_driver import connect

clickhouse_cursor = connect(host='localhost', port='9000', user='default', password='', database='default').cursor()

clickhouse_cursor.execute("""
    CREATE TABLE IF NOT EXISTS movie_views(
        user_id UUID,
        movie_id UUID,
        frame INTEGER,
        event_time DateTime
    )
    ENGINE = MergeTree()
    ORDER BY event_time;
""")

#### *Vertica*

In [ ]:
from vertica_python import Connection

conn_info = {
    'host': 'localhost',
    'port': 5433,
    'user': 'dbadmin',
    'password': 'password',
    'database': 'test'
}

vertica_cursor = Connection(conn_info).cursor()

In [125]:
vertica_cursor.execute("""
    CREATE TABLE IF NOT EXISTS movie_views (
        user_id UUID,
        movie_id UUID,
        frame INTEGER,
        event_time DateTime
    )
    ORDER BY event_time;
""")


#### 2. Функции для генерации данных и вставки в хранилище

In [55]:
from itertools import islice
import random
from faker import Faker

faker = Faker()

def generate_data(num_records, num_users=100, num_movies=100):
    users = [str(faker.uuid4()) for _ in range(num_users)]
    movies = [str(faker.uuid4()) for _ in range(num_movies)]
    
    for _ in range(num_records):
        user_id = random.choice(users)
        movie_id = random.choice(movies)
        frame = faker.random_int(min=0, max=5000)
        event_time = faker.date_time_this_year(before_now=True, after_now=False, tzinfo=None)
        yield user_id, movie_id, frame, event_time

def chunks(generator, size):
    iterator = iter(generator)
    for first in iterator:
        yield [first] + list(islice(iterator, size - 1))

In [14]:
def insert_into_clickhouse(data):
    clickhouse_cursor.execute(
        'INSERT INTO movie_views (user_id, movie_id, frame, event_time) VALUES',
        data
    )
    
def insert_into_vertica(data):
    vertica_cursor.executemany(
        "INSERT INTO movie_views (user_id, movie_id, frame, event_time) VALUES (%s, %s, %s, %s)",
        data
    )

#### 3. Тестирование вставки данных

#### *Clickhouse*

In [35]:
%%time
for data_chunk in chunks(generate_data(10000000), 10000):
    insert_into_clickhouse(data_chunk)

Всего записей для вставки: 10000000
CPU times: user 2min 46s, sys: 531 ms, total: 2min 46s
Wall time: 3min 18s


In [106]:
clickhouse_cursor.execute('SELECT COUNT(*) FROM movie_views')
clickhouse_cursor.fetchone()

(10006200,)

#### *Vertica*

In [126]:
%%time
for data_chunk in chunks(generate_data(10000000), 10000):
    insert_into_vertica(data_chunk)

CPU times: user 5min 4s, sys: 1.14 s, total: 5min 5s
Wall time: 6min 24s


In [127]:
vertica_cursor.execute('SELECT COUNT(*) FROM movie_views')
vertica_cursor.fetchall()

[[10000000]]

#### 4. Получение агрегированных данных

#### *Clickhouse*

In [113]:
%%time
clickhouse_cursor.execute("""
SELECT 
    user_id, 
    count(movie_id) AS movies_watched, 
    avg(frame) AS average_frames
FROM movie_views
GROUP BY user_id
ORDER BY movies_watched DESC
LIMIT 100;
""")

CPU times: user 33.2 ms, sys: 4.41 ms, total: 37.6 ms
Wall time: 3.74 s


In [114]:
%%time
clickhouse_cursor.execute('SELECT MAX(frame), MIN(frame), SUM(frame), AVG(frame) FROM movie_views')

CPU times: user 1.39 ms, sys: 926 µs, total: 2.32 ms
Wall time: 84.7 ms


#### *Vertica*

In [128]:
%%time
vertica_cursor.execute("""
SELECT 
    user_id, 
    count(movie_id) AS movies_watched, 
    avg(frame) AS average_frames
FROM movie_views
GROUP BY user_id
ORDER BY movies_watched DESC
LIMIT 100;
""")

CPU times: user 1.92 ms, sys: 1.51 ms, total: 3.43 ms
Wall time: 26 s


In [93]:
%%time
vertica_cursor.execute('SELECT MAX(frame), MIN(frame), SUM(frame), AVG(frame) FROM movie_views')

CPU times: user 875 µs, sys: 820 µs, total: 1.7 ms
Wall time: 591 ms


#### 5. Получение данных под нагрузкой

In [148]:
import threading
from time import sleep

stop_thread = False
def insert_data_forever():
    while not stop_thread:
        data_chunk = list(generate_data(1000))
        insert_into_clickhouse(data_chunk)
        insert_into_vertica(data_chunk)
        sleep(1)

data_thread = threading.Thread(target=insert_data_forever)
data_thread.start()

In [149]:
%%time
clickhouse_cursor.execute("""
SELECT 
    user_id, 
    count(movie_id) AS movies_watched, 
    avg(frame) AS average_frames
FROM movie_views
GROUP BY user_id
ORDER BY movies_watched DESC
LIMIT 100;
""")

CPU times: user 1.7 s, sys: 15 ms, total: 1.72 s
Wall time: 2.49 s


In [ ]:
%%time
vertica_cursor.execute("""
SELECT 
    user_id, 
    count(movie_id) AS movies_watched, 
    avg(frame) AS average_frames
FROM movie_views
GROUP BY user_id
ORDER BY movies_watched DESC
LIMIT 100;
""")

In [150]:
stop_thread = True
data_thread.join()